# Fatty Liver Disease (FLD) Study

- alcoholic vs non-alcoholic FLD, short: AFLD vs NAFLD


**Outline**

1. Study on liver disease types:
    1. Fibrosis
    1. Steatosis
    2. Inflammation
    
2. Two data sets with 
    1. clinical markers
    2. proteome information

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.linear_model
import sklearn.ensemble
import xgboost
import ipywidgets as widgets

In [ ]:
import src.utils as utils

In [ ]:
import os
CPUS = os.cpu_count()
RANDOMSTATE = 29
DATAFOLDER = 'processed/ML'

# Explore datasets

Diagnostic comparators (existing best-in-class) biomarkers
- Fibrosis markers: transient elastography, 2-dimensional shear wave elastography, ELF test, FibroTest, FIB4 score, APRI score, Forns score, ProC3
- Inflammation markers: M30=caspase-cleaved cytokeratin-18 fragments, M65=total CK18, AST:ALT ratio, ProC3
- Steatosis: Controlled attenuation parameter

In [ ]:
pd.set_option('max_columns', 9)

files = [file for file in os.listdir(DATAFOLDER) if '.csv' in file]
w_data = widgets.Dropdown(options=files, index=5)

def show_data(file):
    filename = os.path.join(DATAFOLDER, file)
    global data # only here to show-case data for report
    try:
        data = pd.read_csv(filename, index_col='Sample ID')
    except:
        data = pd.read_csv(filename)
    try:
        w_cols.options = list(data.columns)
    except:
        pass
    display(data.head())
out = widgets.interactive_output(show_data, controls={'file':w_data})

widgets.VBox([w_data, out])

In [ ]:
# # Possible Alternative for DropDown 
# from src.widgets import multi_checkbox_widget

# descriptions=data.columns
# w_cols = multi_checkbox_widget(descriptions)
# w_cols

In [ ]:
w_cols = widgets.SelectMultiple(options=list(data.columns))

def show_selected_proteins(columns):
    if len(columns)> 0:
        display(data[list(w_cols.value)])
        print(data[list(w_cols.value)].describe())
    else:
        print('Select proteins')

out_sel = widgets.interactive_output(show_selected_proteins, {'columns': w_cols})
widgets.VBox([w_cols, out_sel])

## Clinical data
### Load Complete clinical data

In [ ]:
PROTEOM  = 'data_ml_proteomics.csv'
CLINICAL = 'df_cli_164.csv'
COL_ID = 'Sample ID'
f_data_clinic = os.path.join(DATAFOLDER, CLINICAL)
data_cli = pd.read_csv(f_data_clinic, index_col=COL_ID)
data_cli.head()

In [ ]:
w_cols_cli = widgets.SelectMultiple(options=list(data_cli.columns))

def show_selected_markers(columns):
    if len(columns)> 0:
        display(data_cli[list(w_cols_cli.value)])
        display(data_cli[list(w_cols_cli.value)].describe())
    else:
        print('Select clinical markers')

out_cli = widgets.interactive_output(show_selected_markers, {'columns': w_cols_cli})
widgets.VBox([w_cols_cli, out_cli])

### Selected Clinical markers

Diagnostic comparators (existing best-in-class) biomarkers
- state-of-the-art (**sor**) Fibrosis markers: 
    - `te`: transient elastography (sona liver scan)
    - `swe`: 2-dimensional shear wave elastography
    - `elf`: ELF test
    - `ft`: FibroTest
    - `fib4`: FIB4 score
    - `apri`: APRI score
    - `forns`: Forns score
    - `p3np`: ProC3
- Inflammation markers:
    - M30=caspase-cleaved cytokeratin-18 fragments
    - M65=total CK18
    - AST:ALT ratio
    - ProC3
- Steatosis: Controlled attenuation parameter

In [ ]:
#sor_fibrosis = ['te', 'swe', 'elf', 'ft', 'fib4', 'apri', 'forns', 'p3np']
sor_fibrosis = ['elf', 'ft', 'fib4', 'apri', 'forns', 'p3np']
data_cli.groupby('kleiner')[sor_fibrosis].count()

In [ ]:
pd.set_option('max_columns', 20)
FEATURES_ML = ['nas_steatosis_ordinal', 'nas_inflam', 'kleiner', 
          'fib4', 'elf', 'ft', 'te', 'swe', 'aar','ast',
          'apri','forns','m30', 'm65', 'meld', 'p3np', 'timp1', 'cap' ]
data_cli[FEATURES_ML].head()

In [ ]:
data_cli.groupby('group2')[FEATURES_ML].count()

### Load proteome data

In [ ]:
pd.set_option('max_column', 12)
f_data_proteom = os.path.join(DATAFOLDER, PROTEOM)
data_ml_proteomics = pd.read_csv(f_data_proteom, index_col=COL_ID )
data_ml_proteomics

All "healthy" patients have no fibrosis score, but the prevalence in the general population of fibrosis is between 6-7% this could be a source of confounding.

In [ ]:
fibrosis =   data_ml_proteomics.fibrosis.fillna("NA")   # ML Data
fibrosis_class = data_cli.kleiner.fillna("NA")

pd.crosstab(
index=fibrosis,
columns=fibrosis_class,
margins=True
)

5 plates of clinical data are not present in proteom data.

In [ ]:
data_cli.fibrosis_class.index.difference(data_ml_proteomics.fibrosis.index)

## Proteome data

Questions:
- How to map a new sample to the protein-groups in the training data?

### Load Protein GeneID Mapping

- UniProtID to Gene name mapping


In [ ]:
key_ProteinID = pd.read_csv(os.path.join(DATAFOLDER, 'ID_matching_key.csv'), 
                            index_col="Protein ID").drop("Unnamed: 0", axis=1)
key_ProteinID.head()

In [ ]:
key_ProteinID.loc['P35858']

### Impute missing features of clinical data:

Using [`sklearn.impute.simpleImputer`](https://scikit-learn.org/stable/modules/impute.html)'s default `'mean'` strategy. 
Alternatively one could replace missing values with zeros on the standardised data to zero mean and standard deviation of one.

In [ ]:
FEATURES_CLINIC = ['ggt', 'alt', 'ast', 'alk', 'mcv', 'iga', 'igg', 'leu', 'glc']
data_cli[FEATURES_CLINIC].head()

In [ ]:
#ToDo

## Classifiers

- Select Classifier by cross-validation using [sklearn functionality](https://scikit-learn.org/stable/model_selection.html#model-selection)

In [ ]:
# Define classifiers
clf_xgbc  = xgboost.XGBClassifier(n_jobs=CPUS-1)
clf_rf    = sklearn.ensemble.RandomForestClassifier(n_estimators=200, random_state=RANDOMSTATE)
clf_lr    = sklearn.linear_model.LogisticRegression(random_state=0, solver='liblinear')
clf_svm   = sklearn.svm.SVC(kernel='linear', C=1)
clf_dict = {'xgboost': clf_xgbc,
           'RF': clf_rf,
           'Logistic': clf_lr,
           'SVM': clf_svm,
           }

## Visualization of data

Look at UMAPs with labels from disease categories.
  - Does the assigned disease correspond to certain groups
 
For clinical data, on could look at a selection of scatter plots in order to see if it is feasible to separate some groups based on two features.

# Models

Different _experimental_ setups for prediction models will be compared. First, for the target **fibrosis**. Fibrosis is reported on a five-point scale from stage F0 to F4.

ML setup binary    | HP  | F0  | F1  | F2  | F3  | F4
--- | --- | ---    | --- | --- | --- | ---
HP-F0-F2 vs F3-F4  | c   | c   | c   | c   | t   | t    
F0-F2 vs F3-F4 (advanced)    |     | c   | c   | c   | t   | t
F0-F1 vs F2-F4 (significant)    |     | c   | c   | t   | t   | t

In the table, c stands for control  and t for target. The clinical relevance is to distinguish different 
stages of disease. The question is wheater one should include a healthy, untested patient cohort can help building a 
classification model, as e.g. for fibrosis the general prevalence in the population is between 6 to 7 percent. Alternatively a _multi-task model_ with having 5 classes/end-points can be fit.


In addition to fibrosis, the endpoints **steatosis** and **inflamation** can be predicted.

target      | Scale   | unique values              | N samples
-----       | --------| ---------------            | -------
fibrosis    | five    | F0, F1, F2, F3, F4         | 
steatosis   | five    | S0, S1, S2, S3, S4         | 
inflamation | seven   | I0, I1, I2, I3, I4, I5, I6 | 


What is population of interest?
- population at risk
- general population (which we do not have as a "random" sample)


## based on proteomics data

In [ ]:
data_ml_proteomics.fibrosis.value_counts(dropna=False)

If the models are trained on the fibrosis data only, on could expect some predictions of fibrosis patients in the untested healthy patient (hp) cohort.

### Healthy vs Fibrosis patients

ToDo: Verify how dependent variable is exactly constructed

In [ ]:
target = data_ml_proteomics['class']
X = data_ml_proteomics.iloc[:, :-2]

In [ ]:
# # sanity check
# disease = (data_cli['group2'] == 'ALD').astype('int64')
# shared = target.index.intersection(data_ml_proteomics.index)
# target.equals(disease.loc[shared])

In [ ]:
from sklearn.model_selection import cross_validate
scoring = ['precision', 'recall', 'f1', 'balanced_accuracy', 'roc_auc'] # how to customize cutoff?

import pandas as pd
def run_cv_binary(clf_dict:dict, X:pd.DataFrame, y:pd.Series, cv=5, 
                  scoring=['precision', 'recall', 'f1', 'balanced_accuracy', 'roc_auc'])-> dict:
    """Run Cross Validation (cv) for binary classification example
    for a set of classifiers.
    
    
    Inputs
    ------
    clf_dict: dict
        Dictionary with keys and scikit-learn classifiers as values.
    X: 2D-array, pd.DataFrame
        Input data
    y: 1D-array, pd.Series
        Targets for classification
    cv: int
        Number of splits for Cross-Validation.
    
    Returns
    -------
    dict: dict with keys of clf_dict and computed results for each run. 
    """
    cv_results = {}
    for key, clf in clf_dict.items(): 
        cv_results[key] = cross_validate(clf, X, y=target, cv=5, scoring=scoring)
        cv_results[key]['num_feat'] = X.shape[-1]
    return cv_results
    

To add
-  [x] Stratification of input data
-  [ ] Recursive feature selection
-  [ ] cutoff determination for binary classification (ROC-Curves, Precision-Recall-Curves)

In [ ]:
cv_results = run_cv_binary(clf_dict, X, y=target)

In [ ]:
def _get_cv_means(results_dict:dict) -> pd.DataFrame:
    """Convert result-dictionary of runs to averaged dataframe of results."""
    cv_means = pd.DataFrame(results_dict)
    cv_means = cv_means.applymap(np.mean)
    return cv_means.T

In [ ]:
_get_cv_means(cv_results)

Using Stratified Splitting is default for [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate).

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv_results = {}
for key, clf in clf_dict.items(): 
    cv_results[key] = cross_validate(clf, X, y=target, groups=target, cv=StratifiedKFold(5), scoring=scoring)
    cv_results[key]['num_feat'] = X.shape[-1]

In [ ]:
_get_cv_means(cv_results)

[Feature selection](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection) based on mutual information

In [ ]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

k_best = SelectKBest(mutual_info_classif, k=10)
k_best.fit(X, y=target)

In [ ]:
mask_selection = k_best.get_support()
X.columns[mask_selection]
# X_new = SelectKBest(mutual_info_classif, k=10).fit_transform(X, y=target)

In [ ]:
results_10_best_feat = run_cv_binary(clf_dict, X.loc[:,mask_selection], y=target)
_get_cv_means(results_10_best_feat)

## based on clinical markers

## Versions

In [ ]:
pip list | grep pandas

In [ ]:
%qtconsole